In [ ]:
DEFECT_TYPE = "island"
APPROXIMATE_POSITION_ON_SURFACE = [0.5, 0.5]  # Position of the defect in crystal coordinates
DISTANCE_Z = 2.0  # Distance of the defect from the surface in Angstrom
CHEMICAL_ELEMENT = "Ni"  # Element to be placed at the site 
MILLER_INDICES = (0,0,1 )  # Miller indices of the surface
SLAB_THICKNESS = 3
# Thickness of the slab in unit cells
VACUUM = 6  # Vacuum thickness in Angstrom
SUPERCELL_MATRIX = [[3, 0, 0], [0, 3, 0], [0, 0, 1]]

In [ ]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))


In [ ]:
from mat3ra.made.tools.convert import to_pymatgen, from_pymatgen
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab
from utils.visualize import visualize_materials as visualize

material = materials[0]
slab_config = SlabConfiguration(
    bulk=material,
    miller_indices=MILLER_INDICES,
    thickness=SLAB_THICKNESS,
    vacuum=VACUUM,
    use_orthogonal_z=True,
    xy_supercell_matrix=SUPERCELL_MATRIX
)
termination = get_terminations(slab_config)[0]
slab = create_slab(slab_config, termination)


pymatgen_structure = to_pymatgen(material)
conv_struct =  pymatgen_structure.to_conventional()
conv_material = from_pymatgen(conv_struct)

visualize([{"material": material, "rotation": "0x"}, {"material": material, "rotation": "-90x"}], repetitions=[1, 1, 1])
visualize([{"material": conv_material, "rotation": "0x"}, {"material": conv_material, "rotation": "-90x"}], repetitions=[1, 1, 1])
visualize([{"material": slab, "rotation": "0x"}, {"material": slab, "rotation": "-90x"}], repetitions=[1, 1, 1])


In [ ]:

from utils.jupyterlite import set_data
from mat3ra.made.tools.build.defect.builders import TerraceSlabDefectBuilder
from mat3ra.made.tools.build.defect.configuration import TerraceSlabDefectConfiguration

clean_material = Material.create(Material.default_config)

slab_config = SlabConfiguration(
    bulk=clean_material,
    miller_indices=(0, 1, 1),
    thickness=6,
    vacuum=3,
    xy_supercell_matrix=[[10, 0], [0, 2]],
    use_orthogonal_z=True,
    use_conventional_cell=False
)
t = get_terminations(slab_config)[0]
slab = create_slab(slab_config, t)

config = TerraceSlabDefectConfiguration(
    crystal=slab,
    cut_direction=[1, 0, 3],
    pivot_coordinate=[0.5, 0.5, 0.5],
    steps_number=1,
)
new_slab = TerraceSlabDefectBuilder().get_material(configuration=config)
visualize([{"material": slab, "rotation": "0x"}, {"material": slab, "rotation": "-90x"}], repetitions=[1, 1, 1])
visualize([{"material": new_slab, "rotation": "0x"}, {"material": new_slab, "rotation": "-90x"}], repetitions=[3, 3, 1])
set_data("materials", [new_slab.to_json()])
print(new_slab.basis.coordinates.values[42])

In [ ]:
from mat3ra.made.tools.convert import from_ase, to_ase
from ase import Atoms
from ase.build import molecule
from ase.neighborlist import neighbor_list
from ase.geometry import get_distances
import numpy as np

# Function to passivate undercoordinated atoms in a structure
def passivate_structure(structure, max_coordination=3, passivant='H', bond_length=1.0, cutoff=1.0):
    """
    Passivates dangling bonds on undercoordinated atoms in the provided structure.

    Args:
    structure (ase.Atoms): The ASE Atoms object to be passivated.
    max_coordination (int): Maximum number of neighbors before an atom is considered fully coordinated.
    passivant (str): Type of atom used to passivate (default is 'H' for hydrogen).
    bond_length (float): Target bond length for the passivant atom.

    Returns:
    ase.Atoms: The modified structure with passivated atoms.
    """
    if cutoff is None:
        cutoff = np.min(structure.cell.lengths())/2
    # Calculate nearest neighbors
    i, j, d = neighbor_list('ijd', structure, cutoff=cutoff)
    
    # Count neighbors for each atom
    neighbor_counts = np.bincount(i, minlength=len(structure))
    
    # Determine undercoordinated atoms
    undercoordinated_indices = np.where(neighbor_counts < max_coordination)[0]
    print(undercoordinated_indices)
    
    # New atoms object to add passivating atoms
    new_atoms = structure.copy()

    for index in undercoordinated_indices:
        # Get positions of neighbors
        neighbors = j[i == index]
        position = structure.positions[index]
        
        # Calculate the direction vector pointing away from neighbors
        direction = np.zeros(3)
        for neighbor in neighbors:
            direction += (position - structure.positions[neighbor])
        
        if np.linalg.norm(direction) > 0:
            direction = direction / np.linalg.norm(direction)  # Normalize the direction vector
        
        # Calculate the position for the new atom
        new_atom_position = position + direction * bond_length
        
        # Add the passivant atom
        new_atoms += Atoms(passivant, positions=[new_atom_position])
    
    return new_atoms

# Example usage:
# Load or define your structure

ase_atoms = to_ase(new_slab)

# Passivate the structure
passivated_structure = passivate_structure(ase_atoms, max_coordination=4, cutoff=2.0)

print(passivated_structure)

ase_material = from_ase(ase_atoms)
ase_passivated_material = from_ase(passivated_structure)
visualize([{"material": ase_material, "rotation": "0x"}, {"material": ase_material, "rotation": "-90x"}], repetitions=[1, 1, 1])
visualize([{"material": ase_passivated_material, "rotation": "0x"}, {"material": ase_passivated_material, "rotation": "-90x"}], repetitions=[1, 1, 1])